## Perguntas

### 1 - Como você explicaria frequência espacial em uma imagem?

Frequência espacial determina a quantidade de vezes que uma componente senoidal se repete em uma determinada unidade de distância, ou seja, ocorre em estruturas periódicas. Geralmente expressa por ciclos por milímitro em processamento digital de imagens, ela determina o nível de clareza que a imagem tem.
Nesse contexto, indica a velocidade de modificação de uma luminosidade (intensidade) em uma direção da imagem.

### 2 - Qual seria o resultado de uma imagem filtrada no domínio do espaço por um filtro de média aritmética simples de mesmas dimensões da imagem (considere que não seja feita preenchimentos (padding) na imagem)?

Seria uma imagem com dimensões 1x1, ou seja, um píxel, contendo a média dos pixeis da imagem original.
Portanto, se tivermos a seguinte imagem representada pela matriz $$A =
\begin{bmatrix}
1 & 2 & 3\\
2 & 2 & 1\\
4 & 2 & 1
\end{bmatrix}$$
teríamos a operação

$$

\frac{\text{SUM}(A)}{\text{QTDPIXELS}(A)}

$$

 que resultaria na imagem:
\begin{bmatrix}
2
\end{bmatrix}

### 3 - Explique os procedimentos necessários para fazer a filtragem no domínio da frequência de uma imagem 512x512 por um filtro de dimensões 3x3, ambos estão no domínio espacial.

1- Tendo uma imagem $f(x, y)$ com dimensões 512 x 512, ou seja, M = N = 512, primeiro precisamos aplicar o padding, expandir a imagem, obtendo-se $P = 2M-1$ e $Q = 2M-1$ que serão as novas dimensões da imagem com o padding. Com a imagem de novas dimensões, formamos a imagem que será utilizada colocando o número de zeros necessários na imagem original.

2 - Como próximo passo, é preciso centralizar a transformada multiplicando $f_p(x, y)$ por $(-1)^{x+y}$. 

3 - Com isso, podemos computar a DFT da imagem deslocada, obtendo-se $F(u,v)$.

4 - O próximo passo é gerar o filtro de tamanho correto para ter uma correspondência com a imagem que será filtrada e assim evitar o erro de wraparound. Para isso, geramos uma função filtro real e simétrica $H(u, v)$ de tamanho P x Q (os mesmos P e Q calculados anteriormente) e com centro em ($\frac{P}{2}, \frac{Q}{2}$).

5 - Continuando, multiplicamos nossa função de filtro pela imagem deslocada, ou seja, $G(u, v) = H(u, v)F(u, v)$.

Com $G(u, v)$, podemos fazer a transformada inversa e trazer a imagem para o dominío espacial novamente. Assim, realizamos a seguinte operação $g_p(x, y) = Re(\mathcal{F})continua$, podemos ver que deslocamos a imagem novamente e pegamos só a parte real da transformada uma vez que a imaginária não nos interessa.

Para finalizar, obtemos nossa imagem resultado extraindo a região M x N do segundo quadrante (superior esquerdo) de $g_p(x, y)$, ou seja, removemos o pad para podermos ter nossa imagem de 512 por 512 novamente.

### 4 - O que é o aliasing e por que ele acontece? Como é possível reduzir os seus efeitos?

Aliasing é um efeito que surge no processo de amostragem e faz com que duas amostras do sinal sejam amostrados com interferência, assim ambos sinais se misturam após o processo de amostragem, introduzindo novas componentes no sinal discreto.

Esse efeito ocorre quando o teorema de Nyquist não é garantindo, isto é, um sinal analógico, limitado em banda, que foi amostrado, pode ser perfeitamente recuperado a partir de uma sequência infinita de amostras, se a taxa de amostragem for maior ou igual à $2*F_m$ amostras por segundo, em que $F_m$ é a maior frequência do sinal original. Caso a amostragem serja menor que esse valor iremos introduzir aliasing no sinal.

No caso de processamento digital de imagem temos o aliasing espacial.
Um processo comumente utilizado para evitar o aliasing é utilizar filtros passa-baixa para que possamos diminuir o o intervalo de frequência do sinal amostrado, entretanto, essa abordagem apenas consegue mitigar os efeitos, uma vez que o sinal apresenta esse efeito não seremos mais capazes de obter o sinal original.  

### 5 - Como é possível reconstruir um sinal contínuo a partir de um sinal amostrado? 

Para reconstruir de maneira única um sinal de tempo contínuo a partir de suas amostras, deve haver uma correspondência única entre as FFTs do sinal de tempo contínuo e o sinal amostrado. Estas FFTs se relacionam de forma única se o processo de amostragem não introduzir aliasing.
Para isso precisamos satisfazer o teorema de Nyquist como já foi discutido acima.

Isto é, com um sinal amostrado podemos calcular a FFT gerando um sinal periódico no domínio da frequência, permitindo-nos isolar uma cópia da FFT da função original contínua para fazer a transformada inversa e encontrar o sinal contínuo original no domínio do tempo.

### 6 - Explique o erro de wraparound e como evitá-lo.

Sabemos que na convolução uma máscara é passada sobre a imagem e temos a soma dois valores da máscara multiplicados com suas correspondências na imagem, tendo o pixel central da máscara como nossa referência.

Entretanto, dememos nos atentar que a convolução no domínio da frequência está sucetível a periocidade da FFT da imagem, isto é, nas bordas da imagem a convolução irá pegar parte da imagem original e uma parte do canto oposto da imagem (devido à cópia da fft), gerando-se o efeito de wraparound.

Um exemplo claro é quando a convolução chega na borda superior da imagem e o wraparound insere valores do canto inferior, podendo-se gerar transições abruptadas de intensidade ou até mesmo artefatos.

Para evitarmos esse efeito podemos simplesmente adicionar um padding, de forma que a convolução não irá sobrepor diferentes cópias da imagem original, para isso fazemos com que uma imagem de tamanho $(M, N)$ tenha um tamanho $(2 \times M-1, 2 \times N-1)$, obtendo-se uma cópia original no canto superior esquerdo e os outros cantos serão preenchidos com zero. Dessa forma, ao terminarmos a convolução devemos recortar a parte introduzida.